In [1]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from os import system
import networkx as nx 
import pandas as pd
import collections
import numpy as np
import pydotplus
import re
import os

In [2]:
def proc_value_v2(lable_str):
    value = [int(re.search(r'value = \[(\d+), (\d+)\]', lable_str).group(x)) for x in range(1,3)]
    prc = max(value) / sum(value) * 100
    return prc

In [3]:
path = 'DATA/'
path_reuslt = 'DATA/RESULT/'
file_name = '202006_data_for_tree.csv'
png_name = 'dtree.png'
dot_name = 'tree.dot'
max_depth = 6

In [4]:
df = pd.read_csv(path + file_name, index_col=0)

In [5]:
print(len(df))
df = df.fillna(0)
df.head()

31332


,customer_id,count_,avg_score,avg_amt,allow_f,count_f,count_change_alert_settings,count_pos_purchase,avg_score_pos_purchase,avg_amt_pos_purchase,...,count_close_account,count_view_statement,count_t_socialatm,avg_score_t_socialatm,avg_amt_t_socialatm,count_t_ent_more_30,count_t_s_less_1,sum_term_nm,sum_pay_money,target
0,PF12320193,173,184,104456.0,2,2,0,29,38,4955,...,0,12,0,0,0,105,7,14,2,1
1,OW28028647,46,210,940744.0,0,0,0,13,65,74236,...,0,12,0,0,0,19,1,1,0,0
2,TB15704833,155,202,207987.0,1,1,0,67,107,39836,...,0,2,0,0,0,26,6,0,0,1
3,TB22276042,25,256,5000000.0,0,0,0,0,0,0,...,0,8,0,0,0,12,0,0,0,0
4,PF11735814,1,87,0.0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
train, test = train_test_split(df.drop(columns =['customer_id','count_f','allow_f']), 
                               test_size=0.1,
                               random_state=8,
                               shuffle=True)

X_n_train = train.drop(columns = ['target']).copy()
Y_n_train = train['target'].copy()
X_n_test = test.drop(columns = ['target']).copy()
Y_n_test = test['target'].copy()

In [7]:
dectree = tree.DecisionTreeClassifier(random_state=0, max_depth=max_depth, criterion='entropy')
dectree.fit(X_n_train, Y_n_train)

DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=0)

In [8]:
dot_data = tree.export_graphviz(dectree,
                     out_file=None, 
                     feature_names = X_n_train.columns, 
                     class_names = ['NOT_F','F'],
                     filled=True,
                     rounded=True)

pydotplus_graph = pydotplus.graph_from_dot_data(dot_data)
G = nx.nx_pydot.from_pydot(pydotplus_graph)

In [9]:
leaf_nodes = [x for x in G.nodes() if G.out_degree(x)==0 and G.in_degree(x)==1]
# print(leaf_nodes)

In [10]:
target = 'class = F'
target_leaf = [x for x in leaf_nodes if G.nodes[x]['label'].find(target) != -1 
               and proc_value_v2(G.nodes[x]['label']) > 90]
# print(target_leaf)

In [11]:
target_nodes = list()
for x in target_leaf:
    target_nodes += list(nx.all_simple_paths(G, '0', x))[0]
# print(set(target_nodes))

In [12]:
for x in G.nodes:
    if x not in set(target_nodes):
        pydotplus_graph.del_node(x)

In [13]:
pydotplus_graph.write_png(path_reuslt + 'graph_V2.png')

True